> Can you suggest a visualization that addresses the following point?
>
> FishEye analysts want to better visualize changes in corporate structures over time. Create a visual analytics approach that analysts can use to highlight temporal patterns and changes in corporate structures. Examine the most active people and businesses using visual analytics.

To visualize changes in corporate structures over time and highlight the most active entities, consider using the following visual analytics approach:
1. Interactive Temporal Network Graph

    Nodes: Represent persons and organizations.
    Edges: Show relationships (e.g., ownership, employment).
    Time Slider: Allow users to explore changes over specific time periods.
    Color Coding: Differentiate node types (e.g., CEOs, companies) and edge types (e.g., shareholdership, family relationships).
    Size Scaling: Scale node sizes based on activity level (e.g., number of connections or transactions).

2. Heatmap for Activity Analysis

    Axes: Time (x-axis) vs. Entities (y-axis).
    Cells: Color intensity to represent activity levels (e.g., number of transactions or changes in relationships).
    Hover Information: Display detailed info on activities for specific time periods and entities.

3. Temporal Bar Charts

    Bars: Represent the number of changes in corporate structures over time.
    Stacked Bars: Different colors to indicate different types of changes (e.g., new relationships, terminations).

4. Interactive Dashboards

    Combine the above visualizations into a single dashboard.
    Filters: Enable filtering by entity type, relationship type, or specific time frames.
    Drill-Down Features: Click on nodes or bars to see detailed subgraphs or lists of activities.

5. Summary Tables and Lists

    Highlight the most active people and businesses based on the number of transactions and changes over time.

Tools:

    NetworkX: For network graph creation and manipulation.
    Plotly/D3.js: For interactive and dynamic visualizations.
    Dash/Streamlit: For creating interactive dashboards.

Example Workflow:

    Data Preprocessing: Load the JSON data into a NetworkX graph object.
    Temporal Analysis: Use time-based attributes to filter and analyze data over different periods.
    Visualization: Create interactive and dynamic visualizations to reveal temporal patterns and active entities.
    Interaction: Implement features that allow analysts to explore data and drill down into specifics.

This approach will help FishEye analysts visually track and analyze corporate structure changes over time, highlighting key patterns and active participants in the network.

> Can you provide code to perform what you suggest as the "Interactive Temporal Network Graph". In particular how to have a Time Slider that is aware of "start_date" and "end_date" at the links?

Certainly! Below is an example code to create an interactive temporal network graph using Python libraries such as NetworkX for graph manipulation and Plotly for visualization. The code includes a time slider that is aware of the start_date and end_date attributes of the edges.

In [2]:
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import json

In [3]:
json_path = './MC3/mc3.json'

In [4]:
# open json file
with open(json_path) as json_file:
    data = json.load(json_file)

In [7]:
# Create a NetworkX graph from the JSON data
G = nx.node_link_graph(data)

In [9]:
# Print the first 5 nodes with their attributes
print("Sample nodes with attributes:")
for node, attrs in list(G.nodes(data=True))[:5]:
    print(f"Node {node}: {attrs}")


Sample nodes with attributes:
Node Abbott, Mcbride and Edwards: {'type': 'Entity.Organization.Company', 'country': 'Uziland', 'ProductServices': 'Unknown', 'PointOfContact': 'Rebecca Lewis', 'HeadOfOrg': 'Émilie-Susan Benoit', 'founding_date': '1954-04-24T00:00:00', 'revenue': 5994.73, 'TradeDescription': 'Unknown', '_last_edited_by': 'Pelagia Alethea Mordoch', '_last_edited_date': '2035-01-01T00:00:00', '_date_added': '2035-01-01T00:00:00', '_raw_source': 'Existing Corporate Structure Data', '_algorithm': 'Automatic Import'}
Node Abbott-Gomez: {'type': 'Entity.Organization.Company', 'country': 'Mawalara', 'ProductServices': 'Furniture and home accessories', 'PointOfContact': 'Michael Lopez', 'HeadOfOrg': 'Honoré Lemoine', 'founding_date': '2009-06-12T00:00:00', 'revenue': 71766.67, 'TradeDescription': 'Abbott-Gomez is a leading manufacturer and supplier of high-quality furniture and home accessories, catering to both residential and commercial clients. Their product line includes a wi

In [8]:
G

In [6]:

# Convert dates to datetime objects
for u, v, d in G.edges(data=True):
    d['start_date'] = pd.to_datetime(d['start_date'])
    if 'end_date' in d:
        d['end_date'] = pd.to_datetime(d['end_date'])
    else:
        d['end_date'] = pd.Timestamp.now()

# Extract all unique dates for the time slider
all_dates = sorted(set(d['start_date'] for _, _, d in G.edges(data=True)).union(
    d['end_date'] for _, _, d in G.edges(data=True)))

# Function to filter edges based on the selected date
def filter_edges(date):
    edges = [(u, v, d) for u, v, d in G.edges(data=True) if d['start_date'] <= date <= d['end_date']]
    return edges

# Create the figure
fig = make_subplots(rows=1, cols=1)

# Initial plot with the first date
initial_date = all_dates[0]
edges = filter_edges(initial_date)
edge_x = []
edge_y = []

for edge in edges:
    x0, y0 = G.nodes[edge[0]]['x'], G.nodes[edge[0]]['y']
    x1, y1 = G.nodes[edge[1]]['x'], G.nodes[edge[1]]['y']
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

fig.add_trace(edge_trace)

# Adding nodes
node_x = []
node_y = []
node_text = []

for node in G.nodes(data=True):
    x, y = node[1]['x'], node[1]['y']
    node_x.append(x)
    node_y.append(y)
    node_text.append(node[0])

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        )),
    text=node_text)

fig.add_trace(node_trace)

# Adding time slider
fig.update_layout(
    sliders=[{
        'steps': [{
            'args': [[str(date)], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}],
            'label': str(date),
            'method': 'animate'
        } for date in all_dates],
        'transition': {'duration': 300},
        'x': 0.1,
        'len': 0.9
    }],
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
)

# Adding frames for the animation
frames = []
for date in all_dates:
    edges = filter_edges(date)
    edge_x = []
    edge_y = []

    for edge in edges:
        x0, y0 = G.nodes[edge[0]]['x'], G.nodes[edge[0]]['y']
        x1, y1 = G.nodes[edge[1]]['x'], G.nodes[edge[1]]['y']
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    frames.append(go.Frame(data=[go.Scatter(x=edge_x, y=edge_y, mode='lines')], name=str(date)))

fig.frames = frames

fig.update_layout(title_text='Interactive Temporal Network Graph', showlegend=False)

fig.show()


KeyError: 'start_date'